In [21]:
import pandas as pd
import numpy as npy
from datetime import datetime
from datetime import date

In [3]:
ufcfightscrap=pd.read_csv('fight_hist.csv',sep=',')

In [50]:
#adding a datetime column for ufcfightscrap
datetimecol=dict()
for i in range(len(ufcfightscrap['fighter'])):
    datetimecol[i]=datetime.strptime(ufcfightscrap.values[i,0], '%B %d, %Y').date()
    
datedict={'datetime':datetimecol.values()}
datedataframe = pd.DataFrame (datedict, columns = ['datetime'])
ufcfightscrap['datetime']=datedataframe

In [53]:
ufcfights=ufcfightscrap[['datetime','result','fighter','opponent','division','method','round',
                         'time','knockdowns','sub_attempts','pass','reversals','takedowns_landed'
                         ,'takedowns_attempts','sig_strikes_landed','sig_strikes_attempts'
                         ,'total_strikes_landed','total_strikes_attempts','head_strikes_landed'
                         ,'head_strikes_attempts','body_strikes_landed','body_strikes_attempts'
                         ,'leg_strikes_landed','leg_strikes_attempts','distance_strikes_landed'
                         ,'distance_strikes_attempts','clinch_strikes_landed','clinch_strikes_attempts'
                         ,'ground_strikes_landed','ground_strikes_attempts']]

ufcfighterscrap=pd.read_csv('fighter_stats.csv',sep=',')

In [237]:
def age(birthDate,day=date.today(),form1='%b %d, %Y',form2='%b %d, %Y'):
    if birthDate=='--':
        aa='unknown'
    elif type(birthDate)==str and not type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif type(birthDate)==str and type(day)==str:
        bd=datetime.strptime(birthDate, form1)
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    elif not type(birthDate)==str and type(day)==str:
        bd=birthDate
        today = datetime.strptime(day, form2)
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    else:
        bd=birthDate
        today = day
        aa = today.year - bd.year - ((today.month, today.day) < (bd.month, bd.day)) 
    return aa

#adding an age column to ufcfighterscrap
agecol=dict()
for i in range(len(ufcfighterscrap['dob'])):
    if ufcfighterscrap.values[i,0]=='--':
        agecol[i]='unknown'
    else:
        agecol[i]=age(ufcfighterscrap.values[i,0])
        
agedict={'age':agecol.values()}
agedataframe = pd.DataFrame (agedict, columns = ['age'])
ufcfighterscrap['age']=agedataframe
ufcfighters=ufcfighterscrap[['name','age','height','stance']]

def fighter_age(fighter,day=date.today(),form1='%b %d, %Y',form2='%b %d, %Y'):
    a=0
    for i in range(len(ufcfighters['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            dob=ufcfighterscrap['dob'][i]
            a=age(dob,day,form1,form2)
            break
    return a

#computes a fighters number of wins in the UFC
def wins(fighter):
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]=='W')
    mask=mask1 & mask2
    return mask.sum()

#computes a fighters number of wins in the UFC before day which is intered in the form 'Mar 20, 1990' by default 
#but can be changed to for example ,form='%B %d, %Y'
def wins_before(fighter,day=date.today(),form='%b %d, %Y'):
    ddd=0
    if day==date.today():
        ddd=day
    else:
        ddd=datetime.strptime(day, form).date()
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]=='W')
    mask3=(ufcfights.values[:,0]<ddd)
    mask=mask1 & mask2 & mask3
    return mask.sum()

#computes a fighters number of losses in the UFC
def losses(fighter):
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]=='L')
    mask=mask1 & mask2
    return mask.sum()

#computes a fighters number of losses in the UFC before date
def losses_before(fighter,day=date.today(),form='%b %d, %Y'):
    ddd=0
    if day==date.today():
        ddd=day
    else:
        ddd=datetime.strptime(day, form).date()
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]=='L')
    mask3=(ufcfights.values[:,0]<ddd)
    mask=mask1 & mask2 & mask3
    return mask.sum()

#computes a fighters number of ties/NC in the UFC
def ties(fighter):
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]!='W')
    mask3=(ufcfights.values[:,1]!='L')
    mask=mask1 & mask2 & mask3
    return mask.sum()

#computes a fighters number of ties/NC in the UFC before date
def ties_before(fighter,day=date.today(),form='%b %d, %Y'):
    ddd=0
    if day==date.today():
        ddd=day
    else:
        ddd=datetime.strptime(day, form).date()
    mask1=(ufcfights.values[:,2]==fighter)
    mask2=(ufcfights.values[:,1]!='W')
    mask3=(ufcfights.values[:,1]!='L')
    mask4=(ufcfights.values[:,0]<ddd)
    mask=mask1 & mask2 & mask3 & mask4
    return mask.sum()

#gives a fighters record
def record(fighter,date):
    return (wins(fighter), losses(fighter), ties(fighter))

#gives a fighters record
def record_before(fighter,day=date.today(),form='%b %d, %Y'):
    return (wins_before(fighter,day,form), losses_before(fighter,day,form), ties_before(fighter,day,form))

#functions for height and reach
def fighter_height(fighter):
    a=0
    for i in range(len(ufcfighters['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['height'][i]
            break
    if a=='--':
        b='unknown'
    elif a[4]=='"':
        b=int(a[0])*30.48+int(a[3])*2.54
    else:
        b=int(a[0])*30.48+int(a[3]+a[4])*2.54
    return b

def fighter_reach(fighter):
    a=0
    for i in range(len(ufcfighters['name'])):
        if ufcfighterscrap['name'][i]==fighter:
            a=ufcfighterscrap['reach'][i]
            break
    if a=='--':
        b='unknown'
    else:
        b=int(a[0]+a[1])*2.54
    return b

In [ ]:
#the following column adding procedures can be done in one line as implemented later
#adding a datetime column for ufcfights
#need to make 2065 scale with time
datetimecol=dict()
for i in range(11424):
    datetimecol[i]=datetime.strptime(ufcfightscrap.values[i,0], '%B %d, %Y').date()
    
datedict={'datetime':datetimecol.values()}
datedataframe = pd.DataFrame (datedict, columns = ['datetime'])

ufcfightscrap['datetime']=datedataframe

#setting up columns for rec,wins,losses,ties (can be done in one line each as we do later with other stats)

fighter1reccol=dict()
for i in range(len(ufcfightscrap['fighter'])):
    fighter1reccol[i]=record_before(ufcfightscrap.values[i,4],ufcfightscrap.values[i,0],'%B %d, %Y')
fighter1recdict={'fighter1rec':fighter1reccol}
fighter1recdataframe = pd.DataFrame (fighter1recdict, columns = ['fighter1rec'])

fighter2reccol=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        fighter2reccol[i]=fighter1reccol[i+1]
    else:
        fighter2reccol[i]=fighter1reccol[i-1]
fighter2recdict={'fighter2rec':fighter2reccol}
fighter2recdataframe = pd.DataFrame (fighter2recdict, columns = ['fighter2rec'])

fighter1winscol=dict()
for i in range(11424):
    fighter1winscol[i]=wins_before(ufcfightscrap.values[i,4],ufcfightscrap.values[i,0],'%B %d, %Y')
fighter1winsdict={'fighter1wins':fighter1winscol}
fighter1winsdataframe = pd.DataFrame (fighter1winsdict, columns = ['fighter1wins'])

fighter1lossescol=dict()
for i in range(11424):
    fighter1lossescol[i]=losses_before(ufcfightscrap.values[i,4],ufcfightscrap.values[i,0],'%B %d, %Y')
fighter1lossesdict={'fighter1losses':fighter1lossescol}
fighter1lossesdataframe = pd.DataFrame (fighter1lossesdict, columns = ['fighter1losses'])

fighter1tiescol=dict()
for i in range(11424):
    fighter1tiescol[i]=ties_before(ufcfightscrap.values[i,4],ufcfightscrap.values[i,0],'%B %d, %Y')
fighter1tiesdict={'fighter1losses':fighter1tiescol}
fighter1tiesdataframe = pd.DataFrame (fighter1tiesdict, columns = ['fighter1losses'])

fighter2winscol=dict()
for i in range(11424):
    if i%2==0:
        fighter2winscol[i]=fighter1winscol[i+1]
    else:
        fighter2winscol[i]=fighter1winscol[i-1]
fighter2winsdict={'fighter2wins':fighter2winscol}
fighter2winsdataframe = pd.DataFrame (fighter2winsdict, columns = ['fighter2wins'])

fighter2lossescol=dict()
for i in range(11424):
    if i%2==0:
        fighter2lossescol[i]=fighter1lossescol[i+1]
    else:
        fighter2lossescol[i]=fighter1lossescol[i-1]
fighter2lossesdict={'fighter2losses':fighter2lossescol}
fighter2lossesdataframe = pd.DataFrame (fighter2lossesdict, columns = ['fighter2losses'])

fighter2_ties_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        fighter2_ties_col[i]=fighter1_ties_col[i+1]
    else:
        fighter2_ties_col[i]=fighter1_ties_col[i-1]
fighter2tiesdict={'fighter2ties':fighter2lossescol}
fighter2tiesdataframe = pd.DataFrame (fighter2tiesdict, columns = ['fighter2ties'])

#building columns for age
fighteragecol=dict()
for i in range(11424):
    fighteragecol[i]=fighter_age(ufcfightscrap['fighter'][i], ufcfightscrap['date'][i],'%b %d, %Y','%B %d, %Y')
    
fighteragedict={'fighterage':fighteragecol.values()}
fighteragedataframe = pd.DataFrame (fighteragedict, columns = ['fighterage'])

opponentagecol=dict()
for i in range(11424):
    if i%2==0:
        opponentagecol[i]=fighteragecol[i+1]
    else:
        opponentagecol[i]=fighteragecol[i-1]
opponentagedict={'opponentage':opponentagecol}
opponentagedataframe = pd.DataFrame (opponentagedict, columns = ['opponentage'])


In [ ]:
#making columns for rec,wins,losses,ties
ufcfightscrap['fighter_rec']=fighter1recdataframe
ufcfightscrap['opponent_rec']=fighter2recdataframe
ufcfightscrap['fighter_wins']=fighter1winsdataframe
ufcfightscrap['opponent_wins']=fighter2winsdataframe
ufcfightscrap['fighter_losses']=fighter1lossesdataframe
ufcfightscrap['opponent_losses']=fighter2lossesdataframe
ufcfightscrap['fighter_ties']=fighter1tiesdataframe
ufcfightscrap['opponent_ties']=fighter2tiesdataframe
#making columns for age
ufcfightscrap['fighter_age']=fighteragedataframe
ufcfightscrap['opponent_age']=opponentagedataframe
#making columns for height and reach
ufcfightscrap['fighter_height']=ufcfightscrap['fighter'].apply(fighter_height)
ufcfightscrap['opponent_height']=ufcfightscrap['opponent'].apply(fighter_height)
ufcfightscrap['fighter_reach']=ufcfightscrap['fighter'].apply(fighter_reach)
ufcfightscrap['opponent_reach']=ufcfightscrap['opponent'].apply(fighter_reach)

#still haven't incorporated stance, 
#but honestly I'm not sure how much that would add to the model. We can add it and see if it increases accuracy

In [432]:
#First attempt at a model with only 

ufcfightsML=ufcfightscrap[['date','fighter','opponent','fighter_age','opponent_age','fighter_height',
                           'opponent_height','fighter_reach','opponent_reach',
                           'fighter_wins','opponent_wins','fighter_losses',
                           'opponent_losses','fighter_ties','opponent_ties']]

#we are forming a mask which gives True only when all features are known
df=ufcfightscrap
dff=ufcfightsML

mask1=dff['fighter_age'] != 'unknown'
mask2=dff['opponent_age'] != 'unknown'
mask3=dff['fighter_height'] != 'unknown'
mask4=dff['opponent_height'] != 'unknown'
mask5=dff['fighter_reach'] != 'unknown'
mask6=dff['opponent_reach'] != 'unknown'
mask7=dff['fighter_wins'] != 'unknown'
mask8=dff['opponent_wins'] != 'unknown'
mask9=dff['fighter_losses'] != 'unknown'
mask10=dff['opponent_losses'] != 'unknown'
mask=mask1&mask2&mask3&mask4&mask5&mask6&mask7&mask8&mask9&mask10

ufcfightsML_known=ufcfightsML.values[mask]
ufcfightsML_known_df = pd.DataFrame (ufcfightsML_known, columns = ['date','fighter','opponent','fighter_age','opponent_age','fighter_height','opponent_height','fighter_reach',
                  'opponent_reach','fighter_wins','opponent_wins','fighter_losses','opponent_losses','fighter_ties','opponent_ties'])

#setting up the feature array X and target y
XXX=ufcfightscrap[['fighter_age','opponent_age','fighter_height','opponent_height','fighter_reach',
                  'opponent_reach','fighter_wins','opponent_wins','fighter_losses','opponent_losses']]
XX=XXX.values
X=XX[mask]

yy=ufcfightscrap['result']=='W'
y=yy.values[mask]

#importing sklearn, the machine learning module in python. Then we set up a Logistic Regression model
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X,y)

model.score(X,y)

0.5807006232538148

In [ ]:
#day1 should be input in the form '%B %d, %Y' i.e. 'August 20, 1962'
#conversions can be made via day=datetime.strptime(ufcfightsML_known_df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
def time_diff(day1,day2=date.today()):
    if day2==date.today():
        answer=(day2-datetime.strptime(day, '%B %d, %Y')).days
    else:
        answer=(datetime.strptime(day2, '%B %d, %Y')-datetime.strptime(day1, '%B %d, %Y')).days
    return answer

#we now vectorize this to use in pandas/numpy
time_diff_vect= npy.vectorize(time_diff)

#converts 'August 15,2019' to 'Aug 15,2019'
def convert_date_to_abbrev(day):
    return datetime.strptime(day, '%B %d, %Y').strftime('%b %d, %Y')

#converts 'Aug 15,2019' to 'August 15,2019'
def convert_date_to_unabbrev(day):
    return datetime.strptime(day, '%b %d, %Y').strftime('%B %d, %Y')

#this makes a column 'date_abbrev' in ufcfightscrap with entries of the form 'Aug 15,2019'
ufcfightscrap['date_abbrev']=ufcfightscrap['date'].apply(convert_date_to_abbrev)

#this makes a column 'date_unabbrev' in ufcfightscrap with entries of the form 'August 15,2019'
ufcfightscrap['date_unabbrev']=ufcfightscrap['date_abbrev'].apply(convert_date_to_unabbrev)

#can make a single function to do all of these... actually maybe the count function would even work as is

def L5Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L5Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L2Y_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

def L2Y_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

def ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L5Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L2Y_ko_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

def L2Y_ko_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='KO/TKO':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

def sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
    return summ

def L5Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L5Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>1825:
            break
    return summ

def L2Y_sub_wins(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='W' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

def L2Y_sub_losses(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    for i in good_indices:
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['result'].values[i]=='L' and ufcfightscrap['method'].values[i]=='SUB':
            summ+=1
        else:
            summ+=0
        if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>730:
            break
    return summ

#vectorize all of these functions
L5Y_wins_vect= npy.vectorize(L5Y_wins)
L5Y_losses_vect= npy.vectorize(L5Y_losses)
L2Y_wins_vect= npy.vectorize(L2Y_wins)
L2Y_losses_vect= npy.vectorize(L2Y_losses)
ko_wins_vect= npy.vectorize(ko_wins)
ko_losses_vect= npy.vectorize(ko_losses)
L5Y_ko_wins_vect= npy.vectorize(L5Y_ko_wins)
L5Y_ko_losses_vect= npy.vectorize(L5Y_ko_losses)
L2Y_ko_wins_vect= npy.vectorize(L2Y_ko_wins)
L2Y_ko_losses_vect= npy.vectorize(L2Y_ko_losses)
sub_wins_vect= npy.vectorize(sub_wins)
sub_losses_vect= npy.vectorize(sub_losses)
L5Y_sub_wins_vect= npy.vectorize(L5Y_sub_wins)
L5Y_sub_losses_vect= npy.vectorize(L5Y_sub_losses)
L2Y_sub_wins_vect= npy.vectorize(L2Y_sub_wins)
L2Y_sub_losses_vect= npy.vectorize(L2Y_sub_losses)

In [681]:
#ok lets make a fighter_L5Y_wins column in ufcfightscrap (from this we can quickly get opponent_L5Y_wins column)
ufcfightscrap['fighter_L5Y_wins']=L5Y_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_losses']=L5Y_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_wins']=L2Y_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_losses']=L2Y_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_ko_wins']=ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_ko_losses']=ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_ko_wins']=L5Y_ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_ko_losses']=L5Y_ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_ko_wins']=L2Y_ko_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_ko_losses']=L2Y_ko_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_sub_wins']=sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_sub_losses']=sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_sub_wins']=L5Y_sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L5Y_sub_losses']=L5Y_sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_sub_wins']=L2Y_sub_wins_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])
ufcfightscrap['fighter_L2Y_sub_losses']=L2Y_sub_losses_vect(ufcfightscrap['fighter'],ufcfightscrap['date'])

In [719]:
#sending to csv 
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [717]:
#Next need to add all of these stats for opponents
#After this lets run our model again and see whats our accuracy
#could probably write a function to do this, instead of having the same code repeated with slight changes

opponent_L5Y_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_wins_col[i]=ufcfightscrap['fighter_L5Y_wins'].values[i+1]
    else:
        opponent_L5Y_wins_col[i]=ufcfightscrap['fighter_L5Y_wins'].values[i-1]
opponent_L5Y_wins_dict={'opponent_L5Y_wins':opponent_L5Y_wins_col}
opponent_L5Y_wins_dataframe = pd.DataFrame (opponent_L5Y_wins_dict, columns = ['opponent_L5Y_wins'])

opponent_L5Y_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_losses_col[i]=ufcfightscrap['fighter_L5Y_losses'].values[i+1]
    else:
        opponent_L5Y_losses_col[i]=ufcfightscrap['fighter_L5Y_losses'].values[i-1]
opponent_L5Y_losses_dict={'opponent_L5Y_losses':opponent_L5Y_losses_col}
opponent_L5Y_losses_dataframe = pd.DataFrame (opponent_L5Y_losses_dict, columns = ['opponent_L5Y_losses'])

opponent_L2Y_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_wins_col[i]=ufcfightscrap['fighter_L2Y_wins'].values[i+1]
    else:
        opponent_L2Y_wins_col[i]=ufcfightscrap['fighter_L2Y_wins'].values[i-1]
opponent_L2Y_wins_dict={'opponent_L2Y_wins':opponent_L2Y_wins_col}
opponent_L2Y_wins_dataframe = pd.DataFrame (opponent_L2Y_wins_dict, columns = ['opponent_L2Y_wins'])

opponent_L2Y_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_losses_col[i]=ufcfightscrap['fighter_L2Y_losses'].values[i+1]
    else:
        opponent_L2Y_losses_col[i]=ufcfightscrap['fighter_L2Y_losses'].values[i-1]
opponent_L2Y_losses_dict={'opponent_L2Y_losses':opponent_L2Y_losses_col}
opponent_L2Y_losses_dataframe = pd.DataFrame (opponent_L2Y_losses_dict, columns = ['opponent_L2Y_losses'])

opponent_ko_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_ko_wins_col[i]=ufcfightscrap['fighter_ko_wins'].values[i+1]
    else:
        opponent_ko_wins_col[i]=ufcfightscrap['fighter_ko_wins'].values[i-1]
opponent_ko_wins_dict={'opponent_ko_wins':opponent_ko_wins_col}
opponent_ko_wins_dataframe = pd.DataFrame (opponent_ko_wins_dict, columns = ['opponent_ko_wins'])

opponent_ko_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_ko_losses_col[i]=ufcfightscrap['fighter_ko_losses'].values[i+1]
    else:
        opponent_ko_losses_col[i]=ufcfightscrap['fighter_ko_losses'].values[i-1]
opponent_ko_losses_dict={'opponent_ko_losses':opponent_ko_losses_col}
opponent_ko_losses_dataframe = pd.DataFrame (opponent_ko_losses_dict, columns = ['opponent_ko_losses'])

opponent_L5Y_ko_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_ko_wins_col[i]=ufcfightscrap['fighter_L5Y_ko_wins'].values[i+1]
    else:
        opponent_L5Y_ko_wins_col[i]=ufcfightscrap['fighter_L5Y_ko_wins'].values[i-1]
opponent_L5Y_ko_wins_dict={'opponent_L5Y_ko_wins':opponent_L5Y_ko_wins_col}
opponent_L5Y_ko_wins_dataframe = pd.DataFrame (opponent_L5Y_ko_wins_dict, columns = ['opponent_L5Y_ko_wins'])

opponent_L5Y_ko_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_ko_losses_col[i]=ufcfightscrap['fighter_L5Y_ko_losses'].values[i+1]
    else:
        opponent_L5Y_ko_losses_col[i]=ufcfightscrap['fighter_L5Y_ko_losses'].values[i-1]
opponent_L5Y_ko_losses_dict={'opponent_L5Y_ko_losses':opponent_L5Y_ko_losses_col}
opponent_L5Y_ko_losses_dataframe = pd.DataFrame (opponent_L5Y_ko_losses_dict, columns = ['opponent_L5Y_ko_losses'])

opponent_L2Y_ko_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_ko_wins_col[i]=ufcfightscrap['fighter_L2Y_ko_wins'].values[i+1]
    else:
        opponent_L2Y_ko_wins_col[i]=ufcfightscrap['fighter_L2Y_ko_wins'].values[i-1]
opponent_L2Y_ko_wins_dict={'opponent_L2Y_ko_wins':opponent_L2Y_ko_wins_col}
opponent_L2Y_ko_wins_dataframe = pd.DataFrame (opponent_L2Y_ko_wins_dict, columns = ['opponent_L2Y_ko_wins'])

opponent_L2Y_ko_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_ko_losses_col[i]=ufcfightscrap['fighter_L2Y_ko_losses'].values[i+1]
    else:
        opponent_L2Y_ko_losses_col[i]=ufcfightscrap['fighter_L2Y_ko_losses'].values[i-1]
opponent_L2Y_ko_losses_dict={'opponent_L2Y_ko_losses':opponent_L2Y_ko_losses_col}
opponent_L2Y_ko_losses_dataframe = pd.DataFrame (opponent_L2Y_ko_losses_dict, columns = ['opponent_L2Y_ko_losses'])

opponent_sub_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_sub_wins_col[i]=ufcfightscrap['fighter_sub_wins'].values[i+1]
    else:
        opponent_sub_wins_col[i]=ufcfightscrap['fighter_sub_wins'].values[i-1]
opponent_sub_wins_dict={'opponent_sub_wins':opponent_sub_wins_col}
opponent_sub_wins_dataframe = pd.DataFrame (opponent_sub_wins_dict, columns = ['opponent_sub_wins'])

opponent_sub_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_sub_losses_col[i]=ufcfightscrap['fighter_sub_losses'].values[i+1]
    else:
        opponent_sub_losses_col[i]=ufcfightscrap['fighter_sub_losses'].values[i-1]
opponent_sub_losses_dict={'opponent_sub_losses':opponent_sub_losses_col}
opponent_sub_losses_dataframe = pd.DataFrame (opponent_sub_losses_dict, columns = ['opponent_sub_losses'])

opponent_L5Y_sub_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_sub_wins_col[i]=ufcfightscrap['fighter_L5Y_sub_wins'].values[i+1]
    else:
        opponent_L5Y_sub_wins_col[i]=ufcfightscrap['fighter_L5Y_sub_wins'].values[i-1]
opponent_L5Y_sub_wins_dict={'opponent_L5Y_sub_wins':opponent_L5Y_sub_wins_col}
opponent_L5Y_sub_wins_dataframe = pd.DataFrame (opponent_L5Y_sub_wins_dict, columns = ['opponent_L5Y_sub_wins'])

opponent_L5Y_sub_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L5Y_sub_losses_col[i]=ufcfightscrap['fighter_L5Y_sub_losses'].values[i+1]
    else:
        opponent_L5Y_sub_losses_col[i]=ufcfightscrap['fighter_L5Y_sub_losses'].values[i-1]
opponent_L5Y_sub_losses_dict={'opponent_L5Y_sub_losses':opponent_L5Y_sub_losses_col}
opponent_L5Y_sub_losses_dataframe = pd.DataFrame (opponent_L5Y_sub_losses_dict, columns = ['opponent_L5Y_sub_losses'])

opponent_L2Y_sub_wins_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_sub_wins_col[i]=ufcfightscrap['fighter_L2Y_sub_wins'].values[i+1]
    else:
        opponent_L2Y_sub_wins_col[i]=ufcfightscrap['fighter_L2Y_sub_wins'].values[i-1]
opponent_L2Y_sub_wins_dict={'opponent_L2Y_sub_wins':opponent_L2Y_sub_wins_col}
opponent_L2Y_sub_wins_dataframe = pd.DataFrame (opponent_L2Y_sub_wins_dict, columns = ['opponent_L2Y_sub_wins'])

opponent_L2Y_sub_losses_col=dict()
for i in range(len(ufcfightscrap['fighter'])):
    if i%2==0:
        opponent_L2Y_sub_losses_col[i]=ufcfightscrap['fighter_L2Y_sub_losses'].values[i+1]
    else:
        opponent_L2Y_sub_losses_col[i]=ufcfightscrap['fighter_L2Y_sub_losses'].values[i-1]
opponent_L2Y_sub_losses_dict={'opponent_L2Y_sub_losses':opponent_L2Y_sub_losses_col}
opponent_L2Y_sub_losses_dataframe = pd.DataFrame (opponent_L2Y_sub_losses_dict, columns = ['opponent_L2Y_sub_losses'])


In [ ]:
#adding columns now for opponents stats
ufcfightscrap['opponent_L5Y_wins']=opponent_L5Y_wins_dataframe
ufcfightscrap['opponent_L5Y_losses']=opponent_L5Y_losses_dataframe
ufcfightscrap['opponent_L2Y_wins']=opponent_L2Y_wins_dataframe
ufcfightscrap['opponent_L2Y_losses']=opponent_L2Y_losses_dataframe
ufcfightscrap['opponent_ko_wins']=opponent_ko_wins_dataframe
ufcfightscrap['opponent_ko_losses']=opponent_ko_losses_dataframe
ufcfightscrap['opponent_L5Y_ko_wins']=opponent_L5Y_ko_wins_dataframe
ufcfightscrap['opponent_L5Y_ko_losses']=opponent_L5Y_ko_losses_dataframe
ufcfightscrap['opponent_L2Y_ko_wins']=opponent_L2Y_ko_wins_dataframe
ufcfightscrap['opponent_L2Y_ko_losses']=opponent_L2Y_ko_losses_dataframe
ufcfightscrap['opponent_sub_wins']=opponent_sub_wins_dataframe
ufcfightscrap['opponent_sub_losses']=opponent_sub_losses_dataframe
ufcfightscrap['opponent_L5Y_sub_wins']=opponent_L5Y_sub_wins_dataframe
ufcfightscrap['opponent_L5Y_sub_losses']=opponent_L5Y_sub_losses_dataframe
ufcfightscrap['opponent_L2Y_sub_wins']=opponent_L2Y_sub_wins_dataframe
ufcfightscrap['opponent_L2Y_sub_losses']=opponent_L2Y_sub_losses_dataframe

In [726]:
#Now lets run our model again and see whats our accuracy... hopefully better than .58
#we currently have a two rows for each fight, always with first one the fighter wins and second one the opponent wins
#Lets use the random package to randomly delete one of each pair
import random
random_indices=[]
for i in range(0,len(ufcfightscrap['fighter_wins']),2):
    random_indices.append(random.choice([i,i+1]))

ufc_fights_dirty = pd.read_csv('updated_ufcfightscrap')
ufc_fights_ML_dirty=ufc_fights_dirty[[u'result',u'fighter_wins', u'opponent_wins', u'fighter_losses',
       u'opponent_losses', u'fighter_ties', u'opponent_ties', u'fighter_age',
       u'opponent_age', u'fighter_height', u'opponent_height',
       u'fighter_reach', u'opponent_reach',
       u'fighter_L5Y_wins', u'fighter_L5Y_losses', u'fighter_L2Y_wins',
       u'fighter_L2Y_losses', u'fighter_ko_wins', u'fighter_ko_losses',
       u'fighter_L5Y_ko_wins', u'fighter_L5Y_ko_losses',
       u'fighter_L2Y_ko_wins', u'fighter_L2Y_ko_losses', u'fighter_sub_wins',
       u'fighter_sub_losses', u'fighter_L5Y_sub_wins',
       u'fighter_L5Y_sub_losses', u'fighter_L2Y_sub_wins',
       u'fighter_L2Y_sub_losses', u'opponent_L5Y_wins', u'opponent_L5Y_losses',
       u'opponent_L2Y_wins', u'opponent_L2Y_losses', u'opponent_ko_wins',
       u'opponent_ko_losses', u'opponent_L5Y_ko_wins',
       u'opponent_L5Y_ko_losses', u'opponent_L2Y_ko_wins',
       u'opponent_L2Y_ko_losses', u'opponent_sub_wins', u'opponent_sub_losses',
       u'opponent_L5Y_sub_wins', u'opponent_L5Y_sub_losses',
       u'opponent_L2Y_sub_wins', u'opponent_L2Y_sub_losses']].drop(random_indices)

mask=ufc_fights_ML_dirty['result'] != 'unknown'
for col in ufc_fights_ML_dirty.columns:
    mask1=ufc_fights_ML_dirty[col] != 'unknown'
    mask=mask&mask1
    
ufc_fights_ML=ufc_fights_ML_dirty[mask].drop(['result'], axis=1)

#setting up features and target for the next round of predictions. hopefully we beat .58 accuracy
X=ufc_fights_ML.values
y=(ufc_fights_ML_dirty[mask])['result']=='W'
import sklearn
import scipy
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X,y)
model.score(X,y)

0.7085389401416005

In [766]:
def ufc_prediction_tuple(fighter1,fighter2,day=date.today()):
    return [wins_before(fighter1,day),wins_before(fighter2,day),
            losses_before(fighter1,day),losses_before(fighter2,day),
            ties_before(fighter1,day),ties_before(fighter2,day),
            fighter_age(fighter1,day),fighter_age(fighter2,day),
            fighter_height(fighter1),fighter_height(fighter2),
            fighter_reach(fighter1),fighter_reach(fighter2),
            L5Y_wins(fighter1,day),L5Y_losses(fighter1,day),
            L2Y_wins(fighter1,day),L2Y_losses(fighter1,day),
            ko_wins(fighter1,day),ko_losses(fighter1,day),
            L5Y_ko_wins(fighter1,day),L5Y_ko_losses(fighter1,day),
            L2Y_ko_wins(fighter1,day),L2Y_ko_losses(fighter1,day),
            sub_wins(fighter1,day),sub_losses(fighter1,day),
            L5Y_sub_wins(fighter1,day),L5Y_sub_losses(fighter1,day),
            L2Y_sub_wins(fighter1,day),L2Y_sub_losses(fighter1,day),
            L5Y_wins(fighter2,day),L5Y_losses(fighter2,day),
            L2Y_wins(fighter2,day),L2Y_losses(fighter2,day),
            ko_wins(fighter2,day),ko_losses(fighter2,day),
            L5Y_ko_wins(fighter2,day),L5Y_ko_losses(fighter2,day),
            L2Y_ko_wins(fighter2,day),L2Y_ko_losses(fighter2,day),
            sub_wins(fighter2,day),sub_losses(fighter2,day),
            L5Y_sub_wins(fighter2,day),L5Y_sub_losses(fighter2,day),
            L2Y_sub_wins(fighter2,day),L2Y_sub_losses(fighter2,day)]

In [767]:
#date entered in the form 'Aug 20, 2013'
def ufc_predict(fighter1,fighter2,day=date.today()):
    value=model.predict([ufc_prediction_tuple(fighter1,fighter2,day)])[0]
    if value==True:
        return fighter1
    else:
        return fighter2

In [ ]:
#ANNOYING ISSUE: SOME FUNCTIONS TAKE DATES OF THE FORM 'Aug 20, 2015' and others take only 'August 20, 2015'
#Need to fix this to  make past predictions...
ufc_predict('Frankie Edgar','Pedro Munhoz','Aug 20, 2015')
#running this gives the error time data 'Aug 20, 2015' does not match format '%B %d, %Y'
# and changing date conventions gives another error

In [783]:
ufc_predict('John Dodson','Merab Dvalishvili')

'Merab Dvalishvili'

In [784]:
ufc_predict('Merab Dvalishvili','John Dodson')

'John Dodson'

In [ ]:
#Above is illustrated a real big problem. The prediction should absolutely be independent of the choice of ordering...
#whats going on... Maybe this is only the case for fights with very low confindence levels (close to hyperplane...)?
#we should test how often this happens

In [ ]:
#returns accuracy of the predict function over the last num fights (actually this is something like an upper limit)
#in this prediction we also know the future values of fighter stats (could incorporate date into this but having issue)
df=ufc_fights_dirty.drop(random_indices)[mask]
def ufc_accuracy(num2,num1=0):
    cor=0
    inc=0
    for i in range(2*num1,2*num2,2):
        day=datetime.strptime(df['date'][i], '%B %d, %Y').strftime('%b %d, %Y')
        if df['fighter'][i]==predict(df['fighter'][i],df['opponent'][i],day):
            cor+=1
        else:
            inc+=1
    return float(cor)/(cor+inc)
#AGAIN THIS ANNOYING DATE ISSUE IS COMING UP... FUCK

In [903]:
#enter date unabbreviated 'July 4, 2019'
#here the average gives avg per fight. Later in avg_count we change to average per time spent in octagon
def count(stat, guy,inf_abs, total_L5Y_L2Y_avg, day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    if total_L5Y_L2Y_avg=='total' or total_L5Y_L2Y_avg=='avg':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0]
    elif total_L5Y_L2Y_avg=='L2Y':
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<730 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0]
    else:
        good_indices_1=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date_unabbrev'][i],day1)<1825 and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0]
    if inf_abs=='inf':
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy]
    else:
        good_indices_2=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'].values[i]==guy]
    good_indices=[i for i in good_indices_1 if i in good_indices_2]
    if total_L5Y_L2Y_avg!='avg':
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
    else:
        for i in good_indices:
            summ+=ufcfightscrap[stat][i]
        day1=convert_date_to_abbrev(day1)
        number_fights=wins_before(guy,day1)+losses_before(guy,day1)
        summ=summ/float(number_fights)
    return summ
        
#note a better average is per time not per fight. We will do each stat as an average per one minutes

#enter time in the form 'August 24, 2018'
def time_in_octagon(guy,day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    good_indices=[i for i in ufcfightscrap.index.values if time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0 and ufcfightscrap['fighter'].values[i]==guy] 
    for i in good_indices:
        if ufcfightscrap['time'][i][2]==':':
            summ=int(ufcfightscrap['time'][i][0:2])+int(ufcfightscrap['time'][i][3:])/60.0
        else:
            summ+=5*(ufcfightscrap['round'][i]-1)+int(ufcfightscrap['time'][i][0])+int(ufcfightscrap['time'][i][2:])/60.0
    return summ

#enter date unabbreviated 'July 4, 2019'
#gives takedowns per minute
def avg_count(stat, guy,inf_abs, day1=date.today()):
    if day1==date.today():
        day1=date.today().strftime('%B %d, %Y')
    summ=0
    if inf_abs=='inf':
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['fighter'].values[i]==guy and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0]
    else:
        good_indices=[i for i in ufcfightscrap.index.values if ufcfightscrap['opponent'].values[i]==guy and time_diff(ufcfightscrap['date_unabbrev'][i],day1)>0]
    for i in good_indices:
        summ+=ufcfightscrap[stat][i]
    t= time_in_octagon(guy,day1)
    if t==0:
        summ=0
    else:
        summ=summ/t
    return summ

#vectorize these functions
count_vect= npy.vectorize(count)
avg_count_vect= npy.vectorize(avg_count)

In [1051]:
time ufcfightscrap['fighter_inf_knockdowns_avg']=avg_count_vect('knockdowns',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 18min 37s, sys: 24.2 s, total: 2h 19min 1s
Wall time: 2h 19min 26s


In [1052]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1053]:
time ufcfightscrap['fighter_inf_pass_avg']=avg_count_vect('pass',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 18min 30s, sys: 26 s, total: 2h 18min 56s
Wall time: 2h 19min 23s


In [1054]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1056]:
time ufcfightscrap['fighter_inf_reversals_avg']=avg_count_vect('reversals',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 17min 49s, sys: 25.7 s, total: 2h 18min 14s
Wall time: 2h 18min 31s


In [1057]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1058]:
time ufcfightscrap['fighter_inf_sub_attempts_avg']=avg_count_vect('sub_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 17min 20s, sys: 25.8 s, total: 2h 17min 46s
Wall time: 2h 18min 2s


In [1059]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1060]:
time ufcfightscrap['fighter_inf_takedowns_landed_avg']=avg_count_vect('takedowns_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 18min 11s, sys: 26.7 s, total: 2h 18min 38s
Wall time: 2h 19min 1s


In [1061]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1062]:
time ufcfightscrap['fighter_inf_takedowns_attempts_avg']=avg_count_vect('takedowns_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 38min 43s, sys: 40.4 s, total: 2h 39min 23s
Wall time: 2h 44min 51s


In [1063]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1064]:
time ufcfightscrap['fighter_inf_sig_strikes_landed_avg']=avg_count_vect('sig_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 32min 40s, sys: 42.3 s, total: 2h 33min 22s
Wall time: 2h 36min 42s


In [1065]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1066]:
time ufcfightscrap['fighter_inf_sig_strikes_attempts_avg']=avg_count_vect('sig_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 28min 48s, sys: 33.5 s, total: 2h 29min 22s
Wall time: 2h 31min 30s


In [1067]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1068]:
time ufcfightscrap['fighter_inf_total_strikes_landed_avg']=avg_count_vect('total_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 32min 8s, sys: 40.2 s, total: 2h 32min 48s
Wall time: 2h 36min 13s


In [1069]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1070]:
time ufcfightscrap['fighter_inf_total_strikes_attempts_avg']=avg_count_vect('total_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 22min 52s, sys: 28.2 s, total: 2h 23min 20s
Wall time: 2h 24min 36s


In [1071]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1072]:
time ufcfightscrap['fighter_inf_head_strikes_landed_avg']=avg_count_vect('head_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 45min 10s, sys: 57.3 s, total: 2h 46min 8s
Wall time: 2h 52min 31s


In [ ]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [1073]:
time ufcfightscrap['fighter_inf_head_strikes_attempts_avg']=avg_count_vect('head_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

CPU times: user 2h 49min 13s, sys: 47.9 s, total: 2h 50min 1s
Wall time: 2h 52min 19s


In [1074]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [ ]:
time ufcfightscrap['fighter_inf_body_strikes_landed_avg']=avg_count_vect('body_strikes_landed',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

In [ ]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [ ]:
time ufcfightscrap['fighter_inf_body_strikes_attempts_avg']=avg_count_vect('body_strikes_attempts',ufcfightscrap['fighter'],'inf',ufcfightscrap['date_unabbrev'])

In [ ]:
ufcfightscrap.to_csv('ufcfightscrap.csv', index = False)

In [ ]:
#expect this to take about 2.5 hours per cell... That means to get the whole sheet, we need 220 hours... 
#cutting off at 2011 as planned in the Machine Learning Practice file changes this to 1.8 hours and 154 hours...
#so this would save 3 days of computing.
# at current pace we can do about 8 computations per day, so this should take 11 more days
# with the 2011 cut off pace, this should become 8 days
#update: we will not cut this off at 2011, we will go all the way 

In [ ]:
#Ok lets incorporate the following features _inf_ stands for inflicted
#and _abs_ stands for absorbed

#fighter_inf_knockdown_avg DONE
#fighter_inf_pass_avg DONE
#fighter_inf_reversals_avg DONE
#fighter_inf_sub_attempts_avg DONE
#fighter_inf_takedowns_landed_avg DONE
#fighter_inf_takedowns_attempts_avg DONE
#fighter_inf_sig_strikes_landed_avg RUNNING
#fighter_inf_sig_strikes_attempts_avg RUNNING
#fighter_inf_total_strikes_landed_avg DONE
#fighter_inf_total_strikes_attempts_avg RUNNING
#fighter_inf_head_strikes_landed_avg RUNNING
#fighter_inf_head_strikes_attempts_avg RUNNING
#fighter_inf_body_strikes_landed_avg RUNNING
#fighter_inf_body_strikes_attempts_avg
#fighter_inf_leg_strikes_landed_avg
#fighter_inf_leg_strikes_attempts_avg
#fighter_inf_distance_strikes_landed_avg
#fighter_inf_distance_strikes_attempts_avg
#fighter_inf_clinch_strikes_landed_avg
#fighter_inf_clinch_strikes_attempts_avg
#fighter_inf_ground_strikes_landed_avg
#fighter_inf_ground_strikes_attempts_avg

#fighter_abs_knockdown_avg
#fighter_abs_pass_avg
#fighter_abs_reversals_avg
#fighter_abs_sub_attempts_avg
#fighter_abs_takedowns_landed_avg
#fighter_abs_takedowns_attempts_avg
#fighter_abs_sig_strikes_landed_avg
#fighter_abs_sig_strikes_attempts_avg
#fighter_abs_total_strikes_landed_avg
#fighter_abs_total_strikes_attempts_avg
#fighter_abs_head_strikes_landed_avg
#fighter_abs_head_strikes_attempts_avg
#fighter_abs_body_strikes_landed_avg
#fighter_abs_body_strikes_attempts_avg
#fighter_abs_leg_strikes_landed_avg
#fighter_abs_leg_strikes_attempts_avg
#fighter_abs_distance_strikes_landed_avg
#fighter_abs_distance_strikes_attempts_avg
#fighter_abs_clinch_strikes_landed_avg
#fighter_abs_clinch_strikes_attempts_avg
#fighter_abs_ground_strikes_landed_avg
#fighter_abs_ground_strikes_attempts_avg

In [ ]:
#Ok lets incorporate the following features _inf_ stands for inflicted
#and _abs_ stands for absorbed

#opponent_inf_knockdown_avg
#opponent_inf_pass_avg
#opponent_inf_reversals_avg
#opponent_inf_sub_attempts_avg
#opponent_inf_takedowns_landed_avg
#opponent_inf_takedowns_attempts_avg
#opponent_inf_sig_strikes_landed_avg
#opponent_inf_sig_strikes_attempts_avg
#opponent_inf_total_strikes_landed_avg
#opponent_inf_total_strikes_attempts_avg
#opponent_inf_head_strikes_landed_avg
#opponent_inf_head_strikes_attempts_avg
#opponent_inf_body_strikes_landed_avg
#opponent_inf_body_strikes_attempts_avg
#opponent_inf_leg_strikes_landed_avg
#opponent_inf_leg_strikes_attempts_avg
#opponent_inf_distance_strikes_landed_avg
#opponent_inf_distance_strikes_attempts_avg
#opponent_inf_clinch_strikes_landed_avg
#opponent_inf_clinch_strikes_attempts_avg
#opponent_inf_ground_strikes_landed_avg
#opponent_inf_ground_strikes_attempts_avg

#opponent_abs_knockdown_avg
#opponent_abs_pass_avg
#opponent_abs_reversals_avg
#opponent_abs_sub_attempts_avg
#opponent_abs_takedowns_landed_avg
#opponent_abs_takedowns_attempts_avg
#opponent_abs_sig_strikes_landed_avg
#opponent_abs_sig_strikes_attempts_avg
#opponent_abs_total_strikes_landed_avg
#opponent_abs_total_strikes_attempts_avg
#opponent_abs_head_strikes_landed_avg
#opponent_abs_head_strikes_attempts_avg
#opponent_abs_body_strikes_landed_avg
#opponent_abs_body_strikes_attempts_avg
#opponent_abs_leg_strikes_landed_avg
#opponent_abs_leg_strikes_attempts_avg
#opponent_abs_distance_strikes_landed_avg
#opponent_abs_distance_strikes_attempts_avg
#opponent_abs_clinch_strikes_landed_avg
#opponent_abs_clinch_strikes_attempts_avg
#opponent_abs_ground_strikes_landed_avg
#opponent_abs_ground_strikes_attempts_avg

In [ ]:
#Lets see if we can get the opponent columns running in my office